In [ ]:
pip install transformers sentence-transformers faiss-cpu torch

print('testing')


   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
    --------------------------------------- 0.3/18.2 MB ? eta -:--:--
   - -------------------------------------- 0.8/18.2 MB 1.3 MB/s eta 0:00:14
   -- ------------------------------------- 1.0/18.2 MB 1.3 MB/s eta 0:00:13
   -- ------------------------------------- 1.3/18.2 MB 1.3 MB/s eta 0:00:13
   -- ------------------------------------- 1.3/18.2 MB 1.3 MB/s eta 0:00:13
   ---- ----------------------------------- 1.8/18.2 MB 1.3 MB/s eta 0:00:13
   ---- ----------------------------------- 2.1/18.2 MB 1.3 MB/s eta 0:00:13
   ----- ---------------------------------- 2.4/18.2 MB 1.3 MB/s eta 0:00:13
   ----- ---------------------------------- 2.6/18.2 MB 1.3 MB/s eta 0:00:13
   ------ --------------------------------- 2.9/18.2 MB 1.3 MB/s eta 0:00:13
   ------ --------------------------------- 3.1/18.2 MB 1.3 MB/s eta 0:00:12
   ------- ---------